In [1]:
import os
import cv2
import csv
import zipfile

In [3]:
def resize_crop_and_save_info(input_dir, output_dir, csv_file, crop_size=640, num_crops=4):
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Prepare the CSV file to store image info
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        # Write the header
        writer.writerow(['img_id', 'lat', 'lon', 'city', 'direction', 'time', 'angle', 'filename', 'ori_filename'])

        # Iterate through all files in the input directory
        for filename in os.listdir(input_dir):
            # Check if the file is an image (common formats)
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                image_path = os.path.join(input_dir, filename)
                
                # Extract information from filename
                parts = filename.split('_')
                name = parts[0]
                lon = parts[1].split(',')[0]
                lat = parts[1].split(',')[1]
                time = parts[2]
                angle = parts[3].split('.')[0]  # Extract angle before file extension

                # Load the image
                image = cv2.imread(image_path)
                height, width, channels = image.shape

                # Resize the image to make height equal to crop_size
                scale = crop_size / height
                new_width = int(width * scale)
                resized_image = cv2.resize(image, (new_width, crop_size), interpolation=cv2.INTER_AREA)

                # Calculate the step size to get the desired number of crops
                step_size = (new_width - crop_size) // (num_crops - 1)

                # Create the specified number of overlapping crops
                for i in range(num_crops):
                    x_start = i * step_size
                    x_end = x_start + crop_size
                    crop = resized_image[0:crop_size, x_start:x_end]

                    # Save the cropped image with a modified filename
                    crop_filename = f"{filename.split('.')[0]}_{i}.jpg"
                    crop_filepath = os.path.join(output_dir, crop_filename)
                    cv2.imwrite(crop_filepath, crop)

                    # Write image info to CSV
                    writer.writerow([name, lat, lon, 'CD', i, time, angle, crop_filename, filename])

                # print(f"Processed image '{filename}' into {num_crops} overlapping sections.")

In [4]:
input_folder = '/Users/wenlanzhang/Downloads/PhD_UCL/Data/UNDatathon/SVI/chengde/'
output_folder = '/Users/wenlanzhang/Downloads/PhD_UCL/Data/UNDatathon/SVI/Clip_CD/'
csv_output_file = '/Users/wenlanzhang/Downloads/PhD_UCL/Data/UNDatathon/SVI/Clip_CD/image_info.csv'

resize_crop_and_save_info(input_folder, output_folder, csv_output_file, crop_size=640, num_crops=4)


Processed image '1181_117.9170093,41.0100463889_202008_102.408.jpg' into 4 overlapping sections.
Processed image '140882_117.956738912,40.9311448286_202008_187.375.jpg' into 4 overlapping sections.
Processed image '55_117.758659816,41.9512403651_202007_263.541.jpg' into 4 overlapping sections.
Processed image '159921_117.332691942,40.9377664976_202008_246.701.jpg' into 4 overlapping sections.
Processed image '139840_118.667902798,41.0085573977_202008_293.443.jpg' into 4 overlapping sections.
Processed image '273256_117.735138063,41.3154560747_202007_11.248.jpg' into 4 overlapping sections.
Processed image '282826_116.631210441,41.2299369505_202008_55.028.jpg' into 4 overlapping sections.
Processed image '141084_117.951353698,40.9527548959_202008_350.042.jpg' into 4 overlapping sections.
Processed image '94_117.765826574,41.9351804085_202007_265.83.jpg' into 4 overlapping sections.
Processed image '140852_117.95605489,40.9276171167_202008_191.76.jpg' into 4 overlapping sections.
Process

# Select images

In [6]:
import pandas as pd
import os
import shutil

In [5]:
# Load the CSV file
csv_file_path = '/Users/wenlanzhang/Downloads/PhD_UCL/Data/UNDatathon/SVI/CD_image_info.csv'
df = pd.read_csv(csv_file_path)

# Filter rows where 'yolo8' is not 0
non_zero_yolo8 = df[df['yolo_num'] != 0]
non_zero_yolo8

,img_id,lat,lon,city,direction,time,angle,filename,ori_filename,yolo_conf,yolo_bbox,yolo_num
88,273337,41.319894,117.716615,CD,0,202007,26,273337_117_0.jpg,"273337_117.716614714,41.3198936219_202007_26.0...",[0.38361096382141113],"[[0.1628570556640625, 522.3486328125, 90.24436...",1
217,282958,41.208417,116.648275,CD,1,202008,298,282958_116_1.jpg,"282958_116.648274587,41.2084171481_202008_298....",[0.28761547803878784],"[[0.72100830078125, 148.26290893554688, 134.30...",1
219,282958,41.208417,116.648275,CD,3,202008,298,282958_116_3.jpg,"282958_116.648274587,41.2084171481_202008_298....",[0.259611576795578],"[[368.87457275390625, 45.558528900146484, 635....",1
251,163972,40.762869,118.152701,CD,3,202008,350,163972_118_3.jpg,"163972_118.152700626,40.7628691918_202008_350....",[0.3040286600589752],"[[96.63839721679688, 516.8062744140625, 190.68...",1
257,273515,41.328836,117.735764,CD,1,202007,192,273515_117_1.jpg,"273515_117.735764154,41.3288357586_202007_192....",[0.26199549436569214],"[[183.583740234375, 540.1617431640625, 638.652...",1
...,...,...,...,...,...,...,...,...,...,...,...,...
23614,282216,41.233385,116.619810,CD,2,202008,147,282216_116_2.jpg,"282216_116.619809797,41.2333852261_202008_147....",[0.2970636487007141],"[[186.54678344726562, 490.049072265625, 472.52...",1
23635,282984,41.215982,116.636466,CD,3,202008,351,282984_116_3.jpg,"282984_116.636465694,41.2159820889_202008_351....",[0.41998350620269775],"[[423.1917724609375, 501.43719482421875, 527.9...",1
23660,4501,40.951625,117.728557,CD,0,202008,96,4501_117_0.jpg,"4501_117.728557362,40.9516246145_202008_96.796...",[0.314005970954895],"[[371.4073486328125, 0.0, 638.565673828125, 36...",1
23665,212548,41.961809,117.754057,CD,1,202007,338,212548_117_1.jpg,"212548_117.754057295,41.9618088341_202007_338....",[0.2690100371837616],"[[106.37094116210938, 555.3864135742188, 305.8...",1


In [7]:
def copy_images_with_detections(csv_file, source_folder, destination_folder):
    # Load the CSV file into a DataFrame
    df = pd.read_csv(csv_file)

    # Filter rows where 'yolo_num' is not 0
    detected_images = df[df['yolo_num'] != 0]['filename'].tolist()

    # Create the destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Iterate through the filtered images
    for filename in detected_images:
        source_path = os.path.join(source_folder, filename)
        destination_path = os.path.join(destination_folder, filename)

        # Check if the source image exists and copy it to the destination folder
        if os.path.exists(source_path):
            shutil.copy(source_path, destination_path)
            # print(f"Copied: {filename} to {destination_folder}")
        else:
            print(f"File not found: {filename}")

    print(f"Copying completed. {len(detected_images)} images copied to {destination_folder}")

# Example usage
csv_file_path = '/Users/wenlanzhang/Downloads/PhD_UCL/Data/UNDatathon/SVI/CD_image_info.csv'
source_folder = '/Users/wenlanzhang/Downloads/PhD_UCL/Data/UNDatathon/SVI/Clip_CD/'
destination_folder = '/Users/wenlanzhang/Downloads/PhD_UCL/Data/UNDatathon/SVI/Yolo_detected/'
copy_images_with_detections(csv_file_path, source_folder, destination_folder)


Copied: 273337_117_0.jpg to /Users/wenlanzhang/Downloads/PhD_UCL/Data/UNDatathon/SVI/Yolo_detected/
Copied: 282958_116_1.jpg to /Users/wenlanzhang/Downloads/PhD_UCL/Data/UNDatathon/SVI/Yolo_detected/
Copied: 282958_116_3.jpg to /Users/wenlanzhang/Downloads/PhD_UCL/Data/UNDatathon/SVI/Yolo_detected/
Copied: 163972_118_3.jpg to /Users/wenlanzhang/Downloads/PhD_UCL/Data/UNDatathon/SVI/Yolo_detected/
Copied: 273515_117_1.jpg to /Users/wenlanzhang/Downloads/PhD_UCL/Data/UNDatathon/SVI/Yolo_detected/
Copied: 273515_117_2.jpg to /Users/wenlanzhang/Downloads/PhD_UCL/Data/UNDatathon/SVI/Yolo_detected/
Copied: 273515_117_3.jpg to /Users/wenlanzhang/Downloads/PhD_UCL/Data/UNDatathon/SVI/Yolo_detected/
Copied: 324_116_3.jpg to /Users/wenlanzhang/Downloads/PhD_UCL/Data/UNDatathon/SVI/Yolo_detected/
Copied: 142769_117_1.jpg to /Users/wenlanzhang/Downloads/PhD_UCL/Data/UNDatathon/SVI/Yolo_detected/
Copied: 2043_117_1.jpg to /Users/wenlanzhang/Downloads/PhD_UCL/Data/UNDatathon/SVI/Yolo_detected/
Copie